# MINT malaria intervention emulator workshop

Exploration of malaria intervention scenarios using the `minte` Python package and Google Colab.


## 1. Setup

This section installs dependencies and imports functions.

Runtime: Python 3 on Google Colab.


In [ ]:
# Install minte and estimint (adjust source if needed)
# If these are already installed in the environment, this cell can be skipped.

try:
    import minte  # type: ignore
except ImportError:
    %pip install minte estimint --quiet


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display

from minte import run_minter_scenarios, create_scenario_plots

pd.set_option("display.max_columns", 50)


## 2. Core functions

Two main functions from `minte` are used in this workshop:

- `run_minter_scenarios`: build and run batches of malaria intervention scenarios, returning prevalence and cases.
- `create_scenario_plots`: quick plotting helper for emulator output.

Scenarios are defined by vectors of parameters such as resistance, net types, IRS, and LSM coverage. Each position in the vectors corresponds to a scenario.


## 3. Example 1 – reference set of scenarios

This example runs a small set of scenarios reflecting different intervention choices and prints the resulting prevalence and case trajectories.

Parameter vectors below define 11 scenarios (no intervention, IRS only, LSM only, and several ITN strategies).


In [ ]:
scenario_tag = [
    "no_intervention",
    "irs_only",
    "lsm_only",
    "py_only_only",
    "py_only_with_lsm",
    "py_pbo_only",
    "py_pbo_with_lsm",
    "py_pyrrole_only",
    "py_pyrrole_with_lsm",
    "py_ppf_only",
    "py_ppf_with_lsm",
]

res = run_minter_scenarios(
    scenario_tag   = scenario_tag,
    res_use        = [0.2] * 11,
    py_only        = [0.3] * 11,
    py_pbo         = [0.2] * 11,
    py_pyrrole     = [0.1] * 11,
    py_ppf         = [0.05] * 11,
    prev           = [0.55] * 11,
    Q0             = [0.92] * 11,
    phi            = [0.85] * 11,
    season         = [0] * 11,
    irs            = [0.4] * 11,
    itn_future     = [0.00, 0.00, 0.00, 0.45, 0.45, 0.45, 0.45, 0.45, 0.45, 0.45, 0.45],
    net_type_future= [None, None, None, "py_only", "py_only", "py_pbo", "py_pbo", "py_pyrrole", "py_pyrrole", "py_ppf", "py_ppf"],
    irs_future     = [0.0, 0.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
    routine        = [0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1],
    lsm            = [0.0, 0.0, 0.2, 0.0, 0.2, 0.0, 0.2, 0.0, 0.2, 0.0, 0.2],
)

print("Prevalence results:")
display(res.prevalence.head())

print("\nCases results:")
display(res.cases.head())

print("\nScenario meta:")
display(res.scenario_meta)


### 3.1 Plotting prevalence trajectories

`create_scenario_plots` generates combined and individual plots for the scenarios.

The code below plots prevalence for all scenarios on a single figure.


In [ ]:
plots_prev = create_scenario_plots(
    results=res.prevalence,
    output_dir=None,
    plot_type="combined",
    predictor="prevalence",
)

plots_prev["combined"]


In [ ]:
plots_cases = create_scenario_plots(
    results=res.cases,
    output_dir=None,
    plot_type="combined",
    predictor="cases",
)

plots_cases["combined"]


## 4. Parameters in `run_minter_scenarios`

Each element of the input vectors defines one scenario. Important arguments:

- `res_use`: current resistance level to pyrethroids (0–1).
- `py_only`, `py_pbo`, `py_pyrrole`, `py_ppf`: proportions of each ITN type in current use (sum can be ≤ 1).
- `prev`: baseline malaria prevalence.
- `Q0`: proportion of bites taken indoors.
- `phi`: proportion of bites taken while in bed under nets.
- `season`: seasonality indicator (0 = low seasonality, 1 = high seasonality).
- `routine`: routine case management indicator.
- `irs`: current IRS coverage.
- `irs_future`: IRS coverage after the next campaign.
- `lsm`: larval source management coverage.
- `itn_future`: future ITN coverage after the next campaign (0–1).
- `net_type_future`: future ITN type (e.g. `"py_only"`, `"py_pbo"`, `"py_pyrrole"`, `"py_ppf"`).
- `scenario_tag`: labels for scenarios (used in plots and tables).

The function returns a `MinterResults` object with:

- `prevalence`: predicted prevalence trajectories.
- `cases`: predicted cases trajectories.
- `scenario_meta`: scenario tags and EIR validity flag.


## 5. Example 2 – sweep future ITN coverage

This example varies future ITN coverage from 0 to 80% for a single setting, keeping other parameters fixed. This illustrates how to explore a one-dimensional slice of parameter space.


In [ ]:
n_scenarios = 9
itn_levels = np.linspace(0.0, 0.8, n_scenarios)

scenario_tag_sweep = [f"itn_{int(level*100):02d}pc" for level in itn_levels]

res_sweep = run_minter_scenarios(
    scenario_tag   = scenario_tag_sweep,
    res_use        = [0.2] * n_scenarios,
    py_only        = [0.3] * n_scenarios,
    py_pbo         = [0.2] * n_scenarios,
    py_pyrrole     = [0.1] * n_scenarios,
    py_ppf         = [0.05] * n_scenarios,
    prev           = [0.55] * n_scenarios,
    Q0             = [0.92] * n_scenarios,
    phi            = [0.85] * n_scenarios,
    season         = [0] * n_scenarios,
    irs            = [0.4] * n_scenarios,
    itn_future     = itn_levels.tolist(),
    net_type_future= ["py_pbo"] * n_scenarios,
    irs_future     = [0.0] * n_scenarios,
    routine        = [1] * n_scenarios,
    lsm            = [0.0] * n_scenarios,
    benchmark      = False,
)

plots_sweep = create_scenario_plots(
    results=res_sweep.prevalence,
    output_dir=None,
    plot_type="combined",
    predictor="prevalence",
)

plots_sweep["combined"]


### 5.1 Summaries over years 3–5

Prevalence trajectories run over multiple years. It is often useful to summarise each scenario by a single number such as mean prevalence in years 3–5.

The code below constructs such a summary for the ITN sweep.


In [ ]:
prev_df = res_sweep.prevalence.copy()

# Keep LSTM model only
prev_df = prev_df[prev_df["model_type"] == "LSTM"].copy()

# Convert timesteps to years (14-day windows, starting at year 0)
window_size = 14
prev_df["years"] = (prev_df["timestep"] * window_size) / 365.0 - 2.0

# Restrict to years 3–5
mask = (prev_df["years"] >= 3.0) & (prev_df["years"] <= 5.0)
prev_df_window = prev_df[mask]

summary = (
    prev_df_window
    .groupby("scenario")["prevalence"]
    .mean()
    .reset_index()
    .rename(columns={"prevalence": "mean_prev_years_3_to_5"})
)

summary


## 6. Example 3 – small 2D grid and heat map

To explore two dimensions of parameter space (for example, resistance and future ITN coverage), construct a grid of scenarios and then summarise each scenario by a single number such as mean prevalence in years 3–5.

Below is a small demonstration with a 6×6 grid.


In [ ]:
res_levels = np.linspace(0.1, 0.6, 6)      # resistance
itn_levels2 = np.linspace(0.0, 0.8, 6)     # future ITN coverage

grid_res_use = []
grid_itn_future = []
grid_tags = []

for r in res_levels:
    for itn in itn_levels2:
        grid_res_use.append(r)
        grid_itn_future.append(itn)
        grid_tags.append(f"res_{r:.2f}_itn_{int(itn*100):02d}pc")

n_grid = len(grid_res_use)

res_grid = run_minter_scenarios(
    scenario_tag   = grid_tags,
    res_use        = grid_res_use,
    py_only        = [0.3] * n_grid,
    py_pbo         = [0.2] * n_grid,
    py_pyrrole     = [0.1] * n_grid,
    py_ppf         = [0.05] * n_grid,
    prev           = [0.55] * n_grid,
    Q0             = [0.92] * n_grid,
    phi            = [0.85] * n_grid,
    season         = [0] * n_grid,
    irs            = [0.4] * n_grid,
    itn_future     = grid_itn_future,
    net_type_future= ["py_pbo"] * n_grid,
    irs_future     = [0.0] * n_grid,
    routine        = [1] * n_grid,
    lsm            = [0.0] * n_grid,
    benchmark      = False,
)

prev_grid = res_grid.prevalence.copy()
prev_grid = prev_grid[prev_grid["model_type"] == "LSTM"].copy()

window_size = 14
prev_grid["years"] = (prev_grid["timestep"] * window_size) / 365.0 - 2.0
mask = (prev_grid["years"] >= 3.0) & (prev_grid["years"] <= 5.0)
prev_grid = prev_grid[mask]

summary_grid = (
    prev_grid
    .groupby("scenario")["prevalence"]
    .mean()
    .reset_index()
    .rename(columns={"prevalence": "mean_prev_years_3_to_5"})
)

# Attach back the (resistance, ITN) coordinates
coords = pd.DataFrame(
    {
        "scenario": grid_tags,
        "res_use": grid_res_use,
        "itn_future": grid_itn_future,
    }
)

summary_grid = summary_grid.merge(coords, on="scenario", how="left")

summary_grid.head()


In [ ]:
pivot = summary_grid.pivot_table(
    index="res_use",
    columns="itn_future",
    values="mean_prev_years_3_to_5",
)

fig, ax = plt.subplots(figsize=(6, 4))
im = ax.imshow(pivot.values, aspect="auto", origin="lower")

ax.set_xticks(range(len(pivot.columns)))
ax.set_xticklabels([f"{c:.1f}" for c in pivot.columns])

ax.set_yticks(range(len(pivot.index)))
ax.set_yticklabels([f"{r:.2f}" for r in pivot.index])

ax.set_xlabel("Future ITN coverage")
ax.set_ylabel("Resistance (res_use)")
ax.set_title("Mean prevalence, years 3–5")

cbar = fig.colorbar(im, ax=ax)
cbar.set_label("Prevalence")

plt.tight_layout()
plt.show()


## 7. Batch runs and CSV export

`run_minter_scenarios` is vectorised over scenarios, so a single call can handle tens or hundreds of scenarios (as long as memory allows). After running a batch, results can be written to CSV and analysed in R or other tools.


In [ ]:
# Export prevalence, cases, and scenario meta for the ITN sweep

res_sweep.prevalence.to_csv("mint_prevalence_itn_sweep.csv", index=False)
res_sweep.cases.to_csv("mint_cases_itn_sweep.csv", index=False)
res_sweep.scenario_meta.to_csv("mint_scenario_meta_itn_sweep.csv", index=False)

print("Wrote CSV files:")
print("  mint_prevalence_itn_sweep.csv")
print("  mint_cases_itn_sweep.csv")
print("  mint_scenario_meta_itn_sweep.csv")


### 7.1 Example: loading results in R

Example code to read the exported CSV files in R:

```r
library(readr)
library(dplyr)

prev <- read_csv("mint_prevalence_itn_sweep.csv")
cases <- read_csv("mint_cases_itn_sweep.csv")
meta <- read_csv("mint_scenario_meta_itn_sweep.csv")

# Simple summary: mean prevalence by scenario
prev_summary <- prev %>%
  filter(model_type == "LSTM") %>%
  group_by(scenario) %>%
  summarise(mean_prev = mean(prevalence), .groups = "drop")
```


## 8. EIR validity flag

The emulator is calibrated for a particular EIR range. `run_minter_scenarios` computes an internal EIR value for each scenario and marks scenarios inside the valid range with `eir_valid = TRUE`.

Example: filter results to EIR-valid scenarios only.


In [ ]:
valid_scenarios = res_sweep.scenario_meta.query("eir_valid == True")["scenario_tag"].tolist()

prev_valid = res_sweep.prevalence[
    res_sweep.prevalence["scenario"].isin(valid_scenarios)
].copy()

plots_valid = create_scenario_plots(
    results=prev_valid,
    output_dir=None,
    plot_type="combined",
    predictor="prevalence",
)

plots_valid["combined"]
